# Criatura Lendária 2 : Aplicação de Algoritmos Genéticos na Otimização de Hiperparâmetros em Modelos de Deep Learning para Predição de Energia de Total e de Formação de Nanopartículas de Cobre

## Trabalho de conclusão de curso da disciplina de Redes Neurais da Ilum Escola De Ciência, em que foi proposto a criação de uma rede neural MLP para algum tema relevante no meio científico.

## Grupo 6: Anna Karen Pinto; Beatriz Borges; Paulo Henrique dos Santos

## Professor: Daniel Cassar
Campinas/2024

Materiais em escala nanométrica exibem características distintas em comparação com materiais em escalas macrométricas, devido a uma série de fatores, incluindo a superfície exposta dos materiais. Em escalas nanométricas, a relação entre área de superfície e volume é amplificada, tornando a superfície de contato proporcionalmente maior em relação ao volume do material. Essa proporção aumentada da superfície confere propriedades aos materiais nanoestruturados. [2]

Os nanomateriais possuem uma variedade de aplicações, incluindo catálise, imagiologia por ressonância magnética e liberação controlada de fármacos. Além disso, processos de modificação superficial podem ser empregados para mitigar os efeitos citotóxicos associados a certos materiais. Essas modificações podem incluir revestimentos ou funcionalizações que tornam a interação com o ambiente biológico mais favorável, reduzindo assim os efeitos adversos. São classificadas como nanométricas partículas com dimensões tipicamente entre 1-100 nm [2][3][4]. Portanto, é fundamental entendermos como a energia total e de formação influencia no produto final e nas características para a qual a nanopartícula será designada, permitindo a implementação de medidas preventivas e o planejamento adequado, incentivando um investimento tecnológico e científico maior nesta área.

Com essas questões, foi feita uma rede neural anteriormente para a predição dos atributos de saída (valor da energia total e de formação da nanopartícula) [12]. Entretanto, observou-se que a métrica RMSE (Root Mean Square Error) utilizada nessa rede neural não era o ideal, informando que a performance deste modelo de redes neurais não está bom/preciso o suficiente.

Uma forma de melhorar estes parâmetros é fazendo uma otimização dos hiperparâmetros do modelo. A ciência que pode ser utilizada para esta problemática é os Algorítmos Genéticos. Essa técnica de busca e otimização basea-se nos princípios da seleção natural e evolução biológica, sendo uma ótima ferramenta para solucionar problemas de otimização.

Os códigos aqui apresetados estão organizados da seguinte forma:
- Importação das bibliotecas necessárias
- Obtenção e Tratamento dos dados
- Aplicação do GA
- Avaliação dos Resultados e Conlusão

## Importação das Bibliotecas

Todas as bibliotecas aqui importadas foram essenciais para a realização do trabalho.

In [1]:
#Import and Treat Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from used_func import vif_selection
#VIF
from sklearn.linear_model import LinearRegression
from collections import deque
import datetime
import pickle
#NN
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
#GA
from used_func import populacao as func_pop
from used_func import selecao_torneio_min as funcao_selecao
from used_func import mutacao_simples as funcao_mutacao
from used_func import funcao_objetivo
from used_func import cruzamento_uniforme as funcao_cruzamento

## Obtenção e Tratamento dos dados

Nesta etapa, os seguintes passos foram tomados:

- Importando o Dataset e transformando em variável
- Separando em treino e teste
- Aplicando o Logaritmo e Normalização pelo máximo absoluto
- Aplicando o VIF

Cada um desses passos possui sua importância que esta explícita logo acima da sua relização em código.

### Importando o Dataset e transformando em variável:

In [2]:
df = pd.read_csv('dataset.csv', delimiter = ';')
df = df.dropna()
df

,ID,T,tau,time,N_total,N_bulk,N_surface,Volume,R_min,R_max,...,q6q6_S14,q6q6_S15,q6q6_S16,q6q6_S17,q6q6_S18,q6q6_S19,q6q6_S20,q6q6_S20+,Total_E,Formation_E
0,3281,923,45,1,133,42,91,1.570000e-27,4.737965,8.103804,...,0,0,0,0,0,0,0,0,-385.20936,85.61064
1,3282,923,45,2,136,44,92,1.600000e-27,4.023433,8.361129,...,0,0,0,0,0,0,0,0,-393.45040,87.98960
2,2291,723,50,1,149,51,98,1.750000e-27,4.571391,7.953961,...,0,0,0,0,0,0,0,0,-441.89002,85.56998
3,2292,723,50,2,151,54,97,1.780000e-27,4.754535,8.400565,...,0,0,0,0,0,0,0,0,-451.64457,82.89543
4,3091,823,50,1,170,67,103,2.000000e-27,5.673022,8.205280,...,0,0,0,0,0,0,0,0,-506.59469,95.20531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1210,523,5,10,19487,16311,3176,2.290000e-25,34.466223,44.887615,...,0,0,0,0,0,0,0,0,-65609.94800,3374.03200
3996,2110,673,5,10,19507,16289,3218,2.300000e-25,34.247301,44.673954,...,0,0,0,0,0,0,0,0,-65392.19100,3662.58900
3997,1010,423,5,10,19509,16181,3328,2.300000e-25,21.874439,46.705838,...,0,0,0,0,0,0,0,0,-66085.15400,2976.70600
3998,1910,573,5,10,19517,16255,3262,2.300000e-25,33.916720,48.546207,...,0,0,0,0,0,0,0,0,-65662.22800,3427.95200


### Separando em treino e teste

As listas representadas abaixo são referentes, respectivamente, aos atributos e targets. Nas 3 primeiras linhas de código definimos essas listas, e nas 3 últimas separamos entre treino e teste, sublistas que possibilitarão o treinamento e teste da rede neural. Ademais, é válido mencionar que alguns dos atributos possuíam valores zero em todos os seus dados, assim, realizamos uma seleção manual destes, e os descartamos. A lista criada abaixo já está livre destes atributos.

In [16]:
#Definição do que serão as features e os targets
X = df[["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]]
y = df[["Total_E", "Formation_E"]]
atributos = ["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]

# Split de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=10)


y_treino = y_treino.dropna()      #Retirada de valores NAN
X_treino = X_treino.dropna()      #Retirada de valores NAN

### Aplicando o Logaritmo e Normalização pelo máximo absoluto

Foi proposto em em sala com o Orientador que nosso dados precisariam passar por duas normalizações principais: logaritmica e pelo máximo absoluto. Entretanto, nossos dados possuíam muitos valores 0, e o resultado do logaritmo de 0 é uma indefinição matemática. Na biblioteca que utilizamos para realizar o nosso logaritmo(linhas de código comentadas abaixo), o resultado do log de 0 é um valor NaN. Por isso, estávamos encontrando problemas ao treinar a rede, uma vez que o dataset X possuía uma quantidade de dados diferente do Y. Dessa forma, o grupo decidiu por não utilizar o logaritmo.

Assim, realizamos apenas a normalização pelo máximo absoluto, que é um procedimento utilizado para escalar os valores de um conjunto de dados de forma que o valor máximo seja 1 (ou -1, dependendo do intervalo escolhido). O processo em si é bastante simples e envolve dividir todos os valores do conjunto de dados pelo valor máximo absoluto encontrado no conjunto.

In [4]:
#[6]

#X_treino_log = np.log(np.array(X_treino))
#X_teste_log = np.log(np.array(X_teste))
#X_treino_log = X_treino_log.dropna()

max_abs_normalizador = MaxAbsScaler()     #Definindo o normalizador
X_treino_log_normalizado = max_abs_normalizador.fit_transform(X_treino)     #Normalizando X
X_teste_log_normalizado = max_abs_normalizador.fit_transform(X_teste)     #Normalizando o Y

### Aplicando o VIF

Nosso dataset era composto por muitos atributos, e uma possível mitigação para este problema seria aplicar um VIF, que significa "Variance Inflation Factor" (Fator de Inflação da Variância). É uma medida estatística usada para diagnosticar multicolinearidade em uma regressão linear. Multicolinearidade ocorre quando duas ou mais variáveis independentes em um modelo de regressão linear estão altamente correlacionadas, o que pode causar problemas na interpretação dos coeficientes de regressão e na precisão das previsões.[7]

Os códigos deste algorítmo foram disponibilizados pelo professor, de um acervo próprio:

In [5]:
X_norm_vif = vif_selection(X_treino_log_normalizado, atributos)     #Aplicação do VIF em nossos dados
X_final, cols, deleted_cols = X_norm_vif      #Separando o resultado do VIF em 3 dados: dataset final, colunas totais e colunas apagadas

Após todas estas etapas, nossos dados estão prontos para serem aplicados na MLP e no GA.

## Aplicação do GA

Para a criação e aplicação da MLP utilizamos como base o caderno da aula de Torch, mininstrada pelo nosso Orientador. Assim, realizamos algumas mudanças para adaptar a MLP ao nosso caso.[8] Utilizamos uma rede com quantidade de camadas padronizada em 2, e utilizamos do GA para variar os otimizadores do módulo Torch e as funções de ativação.

### Funções de Ativação:

1. **Sigmoid**:
$
\sigma(x) = \frac{1}{1 + e^{-x}}
$

2. **ReLU (Rectified Linear Unit)**:
$
   f(x) = \begin{cases}
   x & \text{se } x > 0 \\
   0 & \text{caso contrário}
   \end{cases}
$

3. **Tanh (Hyperbolic Tangent)**:
$
   \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$

### Otimizadores:

1. **Adam (Adaptive Moment Estimation)**:
   Adam é um otimizador que combina as vantagens de dois outros métodos de otimização: AdaGrad e RMSProp. A fórmula de atualização dos parâmetros é:

$
   \theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t
$

   Onde:
$
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$
$
   m_t = \beta_1 m_{t-1} + (1 - \beta_1) \nabla_\theta J(\theta_t)
$
$
   v_t = \beta_2 v_{t-1} + (1 - \beta_2) (\nabla_\theta J(\theta_t))^2
$

   Aqui, \(\eta\) é a taxa de aprendizado, \(\epsilon\) é um termo pequeno para evitar divisão por zero, e \(\beta_1, \beta_2\) são os parâmetros de momento.

2. **SGD (Stochastic Gradient Descent)**:
   SGD é um método simples de otimização que atualiza os parâmetros na direção negativa do gradiente da função de custo:

$
   \theta_{t+1} = \theta_t - \eta \nabla_\theta J(\theta_t)
$

   Onde \(\eta\) é a taxa de aprendizado e \(\nabla_\theta J(\theta_t)\) é o gradiente da função de custo em relação aos parâmetros.

3. **LBFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno)**:
   LBFGS é um algoritmo de otimização de segunda ordem que aproxima a inversa da matriz Hessiana para encontrar a direção de descida:

$
   \theta_{t+1} = \theta_t - H_t \nabla_\theta J(\theta_t)
$

   Onde \(H_t\) é uma aproximação da inversa da matriz Hessiana.

In [6]:
x = torch.tensor(X_final)     #Transformando nossos dados em tensores, formato utilizado pelo Pytorch
y = torch.tensor(np.array(y_treino))
y = y.float()
x = x.float()

NUM_DADOS_DE_ENTRADA = x.shape[1]     #Numero de dados de entradas representado pela quantidade de colunas do dataset

NUM_DADOS_DE_SAIDA = 2     #Número de dados de saída

TAXA_DE_APRENDIZADO = 0.01      #Definindo a Taxa de Aprendizado da MLP

fn_perda = nn.MSELoss()     #Definindo a função Perda

NUM_EPOCAS = 1000     #Definindo o número de épocas

populacao = func_pop(20)     #Definindo a população inicial

NUM_GERACOES = 1000      #Número de Gerações do GA

otimizadores = {      #Definindo o dicionário de Otimizadores
  'sgd': optim.SGD,
  'adam': optim.Adam,
  'lbfgs': optim.LBFGS,
}

TAMANHO_TORNEIO = 5      #Definindo o tamanho do sorteio

chance_de_mutacao = 0.1      #Definindo a chance de mutação

CHANCE_DE_CRUZAMENTO = 0.5      #Definindo a chance de cruzamento

redes = []      #Criando a rede de memória

Uma vez que definimos as variáveis imutáveis, e o dicionário contendo os otimizadores, podemos prosseguir para o GA. Assim, realizamos os passos do GA:

- Seleção
    - Objetivo: Escolher os indivíduos mais aptos da população atual para reprodução.
    - Método: Torneio, em que um número fixo de indivíduos é selecionado aleatoriamente e o mais apto (Menor valor de Fitness, ou RSME) do grupo é escolhido.

- Cruzamento
    - Objetivo: Combinar características de dois ou mais indivíduos para criar novos indivíduos (offspring).
    - Método: Cruzamento Uniforme, onde cada gene do offspring é escolhido aleatoriamente de um dos pais.

- Mutação
    - Objetivo: Introduzir variações nos indivíduos ao alterar aleatoriamente alguns de seus genes.
    - Método: Mutação simples, em que um dos genes aleatoriamente é trocado por outro gene possível naquele locus(Localização do gene).

- Atualização do Hall da fama
    - Objetivo: Manter um registro dos melhores indivíduos encontrados durante todas as gerações.
    - Método: Armazenamento dos Melhores Indivíduos: Após cada geração, verificar se os indivíduos atuais são melhores do que os registrados no Hall da Fama

- Encerramento
    - Objetivo: Determinar quando o GA deve parar, e definir a população como a nova geração.

Para realizar essas ações, utilizamos de funções definidas no Script. A base para a definição dessas funções foram os Notebooks da aula de Redes Neurais e Algoritmos Genéticos da Ilum Escola De Ciência, fornecidos pelo Professor Daniel Cassar, bem como a função do VIV utilizado.[12] 

Vale mencionar, ainda, a função de perda escolhida pelo grupo foi a MSE, que é calculado como a média dos quadrados das diferenças entre os valores previstos pelo modelo e os valores reais dos dados de teste, mas os valores que entram no Fitness do GA são de RMSE, por possuírem uam melhor visuaização.[10]

In [7]:
mse = []
hall_da_fama = []
hall_do_rmse = []

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness, redes = funcao_objetivo(
    populacao, 
    redes, otimizadores, 
    NUM_DADOS_DE_ENTRADA, 
    NUM_DADOS_DE_SAIDA, 
    NUM_EPOCAS, 
    x, 
    y, 
    fn_perda, 
    TAXA_DE_APRENDIZADO) # Seleção dos melhores indivíduos (assumindo uma função funcao_selecao definida)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO
    )
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao(populacao, chance_de_mutacao)

    # Atualização do hall da fama
    fitness, redes = funcao_objetivo(proxima_geracao, redes, otimizadores, NUM_DADOS_DE_ENTRADA, NUM_DADOS_DE_SAIDA, NUM_EPOCAS, x, y, fn_perda, TAXA_DE_APRENDIZADO)
    menor_fitness = min(fitness)
    indice = fitness.index(menor_fitness)
    melhor_individuo = proxima_geracao[indice]
    
    # Adicionar ao hall da fama se não estiver presente
    if melhor_individuo not in hall_da_fama:
        hall_da_fama.append(melhor_individuo)
    if menor_fitness not in hall_do_rmse:
        hall_do_rmse.append(menor_fitness)

    # Encerramento
    populacao = proxima_geracao

print(hall_da_fama , hall_do_rmse)


[['relu', 'sgd', 19, 16], ['sigmoid', 'sgd', 17, 2]] [3087.2217283505893, 2817.4878881727245]


Após a selção de algumas redes promissoras por meio do treinamento dos individuos coma otimização do GA, podemos selecionar o melhor individuo baseado no menor fitness, resgatar essa rede treinada e testar os dados no dataset de teste criado anteriormente.

In [18]:
# Validação
rmse = min(hall_do_rmse)
mlp = None
individuo = None
mse = None

for i, d in enumerate(hall_do_rmse):      #Selecionando o melhor individuo
    if rmse == d:
        indice = i
        individuo = hall_da_fama[indice]

        
for b in redes:      #Selecionando a melhor rede
    if individuo == b['individuo']:
        mlp = b['mlp']
        mse = b['mse']

X_teste = X_teste[cols]
X_teste = np.array(X_teste)
y_teste = np.array(y_teste)        
x_test_tensor = torch.tensor(X_teste)
y_teste_tensor = torch.tensor(y_teste)
x_test_tensor = x_test_tensor.float()
y_teste_tensor = y_teste_tensor.float()

with torch.no_grad():      #Foward Pass
    output = mlp(x_test_tensor)

loss = fn_perda(output, torch.tensor(y_teste_tensor))     #Avaliação da performance
loss = float(loss**(1/2))

print("Loss no conjunto de teste:", loss)
print("Loss no conjunto de treino:", mse)

Loss no conjunto de teste: 17613360.0
Loss no conjunto de treino: 7938238.0


C:\Users\paulo23008\AppData\Local\Temp\ipykernel_8604\634842956.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = fn_perda(output, torch.tensor(y_teste_tensor))     #Avaliação da performance


## Avaliação dos Resultados e Conclusão

### Avaliação dos Resultados

Por meio do resultado do RMSE utilizando os dados de treino e os dados de teste, podemos ver que os valores não estão próximos. Assim, que não se aplica muito bem ao problema, e que certamente pode melhora por meio do treinamento em mais épocas, mas que resolve bem nosso problema. Uma alternativa poderia ser a utilização de uam ferramenta de geração de dados artificiais para ter uma melhor abrangência do problemas.

Após o treinamento e teste de diferentes arquiteturas da Rede Neural MLP para modelagem de nanomateriais, aplicando o Algoritmo Genético para minimização de seus hiperparâmetros, observamos resultados promissores em termos de melhores valores da métrica de RMSE. Utilizamos uma variedade de arquiteturas, ajustando o tipo de função de ativação, tipo de otimizador, número de camadas e número de neurônios em cada camada para encontrar a configuração mais adequada.

Esses resultados validam a eficácia da abordagem de Algoritmos Genéticos para problemas de otimização e sugerem que esta técnica pode ser aplicada com sucesso em uma variedade de problemas relacionados a modelos de predição de materiais em escala nanométrica.

### Conclusão

Nesta revisão, exploramos a importância de analisar a precisão das predições utilizando cada tipo de otimizador, analisando qual seria a melhor otimização para nosso problema: minimização dos hiperparâmetros.

Após obter os dados de métrica RMSE da Rede Neural MLP utilizada anteriormente, destacamos o uso de Algoritmos Genéticos como uma ferramenta de otimização para minimizar essa métrica. Ao aplicar Algoritmos Genéticos para ajustar os hiperparâmetros da rede neural, pode-se explorar de forma eficiente o espaço de busca e encontrar boas configurações que resultem em previsões mais precisas e uma melhor compreensão dos fatores que influenciam as propriedades dos nanomateriais na rede neural. Essa abordagem nos permite não apenas melhorar o desempenho da rede neural, mas também otimizar o design dos nanomateriais para aplicações específicas, maximizando assim o seu potencial em diversas áreas da Ciência e da Tecnologia.

Em suma, a utilização de Algoritmos Genéticos para minimizar a métrica RMSE de uma rede neural aplicada a nanomateriais, em conjunto com a manipulação de datasets estruturados, representa uma abordagem poderosa e interdisciplinar para avançar nosso entendimento e aplicação desses materiais em escala nanométrica. Esperamos que este trabalho inspire mais pesquisas e investimentos na área, levando a avanços significativos e inovações em nanotecnologia e ciência dos materiais.

# Referências

[1] Copper Nanoparticle Data Set. Disponível em: https://data.csiro.au/collection/csiro:42598. Acesso em: 09 abr. 2024.

[2] OS NANOMATERIAIS E A DESCOBERTA DE NOVOS MUNDOS NA BANCADA DO QUÍMICO | Manuel A. Martins e Tito Trindade - Quim. Nova, Vol. 35, No. 7, 1434-1446, 2012. Disponível em: https://www.scielo.br/j/qn/a/P8tgywDnt7nS6tGyHdQ3BCF/. Acesso em: 02 mai. 2024.

[3] Ojha, N. K.; Zyryanov, G. V.; Majee, A.; Charushin, V. N.; Chupakhin, O. N.; Santra, S. Copper nanoparticles as inexpensive and efficient catalyst: A valuable contribution inorganic synthesis. Coordination Chemistry Reviews 2017, 353, 1–57.11.

‌[4] Ssekatawa K, Byarugaba DK, Angwe MK, Wampande EM, Ejobi F, Nxumalo E, Maaza M, Sackey J, Kirabira JB. Phyto-Mediated Copper Oxide Nanoparticles for Antibacterial, Antioxidant and Photocatalytic Performances. Front Bioeng Biotechnol. 2022 Feb 16;10:820218. doi: 10.3389/fbioe.2022.820218. PMID: 35252130; PMCID: PMC8889028.

‌[5] Multilayer perceptron | Wikipedia, the free encyclopedia. Disponível em https://en.wikipedia.org/wiki/Multilayer_perceptron#:~:text=A%20multilayer%20perceptron%20(MLP)%20is,that%20is%20not%20linearly%20separable.. Acesso em: 29 abr. 2024.

[6] https://towardsdatascience.com/genetic-algorithm-an-optimization-approach-dd60e23261e6

[7] https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html

[8] https://online.stat.psu.edu/stat462/node/180/

[9] ATP-303 NN 5.2 - Notebook PyTorch

[10] https://www.deeplearningbook.com.br/funcao-deativacao/#:~:text=ReLU%20é%20a%20função%20de,neurônios%20ativados%20pela%20função%20ReLU.

[11] https://statisticsbyjim.com/regression/mean-squared-error-mse/